## 2. imports and settings


In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline   
# Pipline allows to chain together multiple data transformers and a final estimator into a single, cohesive object.
from sklearn.ensemble import GradientBoostingRegressor   
# GradientBoostingRegressor is an ensamble ML technique that builds a series of decision trees, each aimed at correcting the errors of the previous ones.
from sklearn.metrics import mean_absolute_error, mean_squared_error 

plt.style.use("seaborn-v0_8") 
pd.set_option("display.max_columns", 50) 

## 3. data loading


In [69]:

data_dir = "../data"

orders = pd.read_csv(
    f"{data_dir}/olist_orders_dataset.csv", 
    parse_dates = ["order_purchase_timestamp", "order_approved_at", 
                   "order_delivered_carrier_date", "order_delivered_customer_date", "order_estimated_delivery_date"])

payments = pd.read_csv(f"{data_dir}/olist_order_payments_dataset.csv")
customers = pd.read_csv(f"{data_dir}/olist_customers_dataset.csv")

order_items = pd.read_csv(f"{data_dir}/olist_order_items_dataset.csv")
products = pd.read_csv(f"{data_dir}/olist_products_dataset.csv")

orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


## 4. build order‑level revenue table and attach customers

In [48]:
# keep only delivered orders from orders

delivered_orders = orders[orders["order_status"] == "delivered"].copy() 

# delivered_orders.head()


In [66]:
# extract revenue per order from payments

order_revenue = (payments 
        .groupby("order_id", as_index=False)["payment_value"] 
        .sum() 
        .rename(columns={"payment_value": "order_revenue"}))

# order_revenue.head()


In [71]:
# merge delivered_orders (left) with order revenue (right) 

orders_with_rev = delivered_orders.merge(order_revenue, on="order_id", how="left") 
orders_with_rev["order_revenue"] = orders_with_rev["order_revenue"].fillna(0.0) 

orders_with_rev.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_revenue
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,28.62


In [72]:
# attach customers (right) to the orders_with_rev (left)
orders_with_cust = orders_with_rev.merge(
    customers[["customer_id", "customer_unique_id", "customer_state"]],
    on="customer_id", how="left")

# filter for specific columns and sort wrt time
orders_df = orders_with_cust[["order_id", 
                              "customer_unique_id", 
                              "customer_state", 
                              "order_purchase_timestamp", 
                              "order_revenue"]].copy() 

orders_df = orders_df.sort_values("order_purchase_timestamp").reset_index(drop=True) 

orders_df.head() 


,order_id,customer_unique_id,customer_state,order_purchase_timestamp,order_revenue
0,bfbd0f9bdef84302105ad712db648a6c,830d5b7aaa3b6f1e9ad63703bec97d23,SP,2016-09-15 12:16:38,0.00
1,3b697a20d9e427646d92567910af6d57,32ea3bdedab835c3aa6cb68ce66565ef,SP,2016-10-03 09:44:50,45.46
2,be5bc2f0da14d8071e2d45451ad119d9,2f64e403852e6893ae37485d5fcacdaf,RS,2016-10-03 16:56:50,39.09
3,a41c8759fbe7aab36ea07e038b2d4465,61db744d2f835035a5625b59350c6b63,RS,2016-10-03 21:13:36,53.73
4,d207cc272675637bfed0062edffd0818,8d3a54507421dbd2ce0a1d58046826e0,SP,2016-10-03 22:06:03,133.46


## 5. define cohorts, t_months, and customer summary

In [73]:
# the date of the first order for each customer  
first_orders = ( 
    orders_df 
    .groupby("customer_unique_id", as_index=False)["order_purchase_timestamp"] 
    .min() 
    .rename(columns={"order_purchase_timestamp": "first_order_date"}) 
)

# attach first_orders (right) to the orders_df (left)
orders_df = orders_df.merge(first_orders, on="customer_unique_id", how="left") 


In [ ]:
# cohort month and months since first order 
orders_df["cohort_month"] = orders_df["first_order_date"].dt.to_period("M") 

order_month = orders_df["order_purchase_timestamp"].dt.to_period("M") 
first_month = orders_df["first_order_date"].dt.to_period("M") 
orders_df["t_months"] = ( 
    order_month.astype("int64") - first_month.astype("int64") 
).astype(int) 

orders_df.head() 